In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
# from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import time
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

/Users/ctfchan/Developer/langchain-experiment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load Documents
loader = PyPDFLoader('data/harvesting_alpha.pdf')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
documents = text_splitter.split_documents(documents)
print("Documents size: {doc_len}".format(doc_len=len(documents)))

Documents size: 21


In [3]:

# Define prompt
prompt_template = """
You are a helpful assistant that helps retrieve topics talked about in a podcast transcript
- Your goal is to summarize snippets of a podcast
- Topics include:
  - Themes
  - Business Ideas
  - Interesting Stories
  - Money making businesses
  - Quick stories about people
  - Mental Frameworks
  - Stories about an industry
  - Analogies mentioned
  - Advice or words of caution
  - Pieces of news or current events
- Use the same words and terminology that is said in the podcast
- Do not respond with anything outside of the podcast. If you don't see any topics, say, 'No Topics'
- Do not respond with numbers, just bullet points
- Do not include anything about 'Marketing Against the Grain'
- Only pull topics from the transcript. Do not use the examples

Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

In [4]:
# 3 RPM for free tier.
summaries = []
for i, doc in enumerate(documents):
    if (i != 0 and i % 3 == 0):
        time.sleep(30)
    summary = chain.run([doc])
    summaries.append(summary)
    print(summary)

Brandon Zick, the CIO of Ceres Partners, manages a $1.4 billion fund that invests in farmland. He grew up on a dairy farm in Pennsylvania before working on Wall Street for over a decade. He discusses the demand and supply dynamics, the economics of farming, and the criteria for investing in farms. Zick emphasizes the value and optionality of owning land and dispels the misconception of consolidated land ownership in the U.S. He also highlights the low institutional ownership of farmland.
The majority of land ownership in the US is held by non-farming heirs, estates, and trusts, with a robust rental market for farmland. Farmland is actively operated by family farms, and there is a trend of consolidation as generations move away from farming. Institutional investors have started to enter the farmland market, but the diverse ownership group makes it difficult for a large-scale roll-up of assets. The turnover of farmland ownership is low, and it will take generations for significant change

KeyboardInterrupt: 

In [20]:
summary_docs = [Document(page_content=t) for t in summaries]
summary_prompt_template = """
You are a helpful assistant that helps retrieve topics talked about in a podcast transcript
- You will be given a series of bullet snippet summaries 
- Only pull topics from the transcript. Do not use the examples

Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
summary_prompt_template = PromptTemplate.from_template(summary_prompt_template)

summary_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
summary_chain = load_summarize_chain(summary_llm, chain_type="stuff", prompt=summary_prompt_template)
final_summary = chain.run(summary_docs)


In [21]:
final_summary

'Brandon Zick, the CIO of Ceres Partners, manages a $1.4 billion fund that invests in farmland. He discusses the demand and supply dynamics, the economics of farming, and the ownership of US farmland. Farmland ownership in the US is not consolidated, with about half owned by heirs, estates, and trusts, and 40-45% owned by actual farmers. Institutional ownership is low. Ceres Farms focuses on underwriting deals with farmers and prioritizes working with local tenants. They aim to sell when prices are high, reinvest when prices are attractive, and believe their structure will benefit them in the current cycle.'